# RAG 流程中的 PII 数据脱敏

## 什么是 PII？

**PII（Personally Identifiable Information）** 是指可以识别出特定个人身份的数据：
- 姓名、手机号、身份证号
- 邮箱、地址、银行卡号
- 医疗记录、财务信息等

## RAG 流程中的 6 个风险点

```
用户输入   →  向量检索  →   文档召回 →   拼接上下文   →   LLM生成 →   日志记录
  ↓           ↓            ↓            ↓              ↓         ↓
风险1        风险2         风险3        风险4          风险5      风险6

风险1：用户直接输入 PII（"我的手机号13812345678"）
风险2：PII 被嵌入向量数据库（检索时泄露）
风险3：检索到他人 PII（越权访问）
风险4：Prompt 包含大量 PII（泄露给 LLM）
风险5：模型输出复现 PII（数据泄露）
风险6：日志记录完整对话（合规风险）
```

## 三种脱敏策略

1. **输入脱敏** - 检测用户输入的 PII，替换为占位符
2. **分级脱敏** - 自己的数据（部分脱敏）vs 他人的数据（完全脱敏）
3. **输出检查** - 防止 LLM 输出中泄露 PII

## 🎯 实验目标

通过模拟 RAG 场景，展示不脱敏的危害和脱敏的效果

---

## 环境准备

In [1]:
# 安装并启动 Ollama
import subprocess
import time
import requests
import json
import re

print("⬇️ 安装 Ollama...")
subprocess.run("curl -fsSL https://ollama.com/install.sh | sh", shell=True, check=False)
print("✅ Ollama 安装完成")

print("🚀 启动 Ollama 服务...")
ollama_server = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5)
print("✅ Ollama 运行中")

print("⬇️ 下载 qwen3:8b...")
subprocess.run(["ollama", "pull", "qwen3:8b"], check=False)
print("✅ 模型就绪")

⬇️ 安装 Ollama...
✅ Ollama 安装完成
🚀 启动 Ollama 服务...
✅ Ollama 运行中
⬇️ 下载 qwen3:8b...
✅ 模型就绪


In [2]:
# 核心工具类
OLLAMA_URL = "http://localhost:11434"

class PIIDetector:
    """PII 检测与脱敏工具"""

    def __init__(self):
        # 常见 PII 正则模式
        self.patterns = {
            'phone': r'1[3-9]\d{9}',
            'id_card': r'\d{17}[\dXx]',
            'email': r'[\w\.-]+@[\w\.-]+\.\w+',
        }

    def detect_and_mask(self, text):
        """完全脱敏：替换为占位符"""
        pii_map = {}
        masked_text = text

        for pii_type, pattern in self.patterns.items():
            for match in re.finditer(pattern, text):
                pii_value = match.group()
                placeholder = f"<{pii_type.upper()}_{len(pii_map)}>"
                pii_map[placeholder] = {'value': pii_value, 'type': pii_type}
                masked_text = masked_text.replace(pii_value, placeholder)

        return masked_text, pii_map

    def partial_mask(self, text):
        """部分脱敏：保留部分字符帮助识别"""
        # 手机号：138****5678
        text = re.sub(r'(\d{3})\d{4}(\d{4})', r'\1****\2', text)
        # 邮箱：abc***@example.com
        text = re.sub(r'([\w]{1,3})[\w\.-]*@', r'\1***@', text)
        return text

def get_completion(prompt, show_output=True):
    """调用 LLM"""
    url = f"{OLLAMA_URL}/api/chat"
    payload = {
        "model": "qwen3:8b",
        "messages": [{"role": "user", "content": prompt}],
        "stream": True,
        "options": {"temperature": 0.7}
    }

    result = ""
    if show_output:
        print("", end='', flush=True)

    try:
        response = requests.post(url, json=payload, stream=True)
        response.raise_for_status()

        for line in response.iter_lines():
            if line:
                try:
                    data = json.loads(line.decode('utf-8'))
                    if 'message' in data and 'content' in data['message']:
                        content = data['message']['content']
                        if content:
                            if show_output:
                                print(content, end='', flush=True)
                            result += content
                    if data.get('done', False):
                        break
                except json.JSONDecodeError:
                    continue

        if show_output:
            print()
        return result.strip()

    except Exception as e:
        if show_output:
            print(f"\n❌ 错误: {str(e)}")
        return f"[错误: {str(e)}]"

detector = PIIDetector()
print("✅ 工具就绪: PIIDetector + get_completion")

✅ 工具就绪: PIIDetector + get_completion


---

# 实验一：用户输入的 PII 检测与脱敏

**场景**：用户在查询时直接输入了敏感信息

**风险**：PII 被发送给 LLM，可能泄露到日志或训练数据

In [3]:
# 实验一：用户输入 PII 检测

user_input = "我叫张三，手机号13812345678，邮箱zhangsan@example.com，请帮我查询订单状态"

print("="*80)
print("实验一：用户输入的 PII 检测与脱敏")
print("="*80)
print(f"原始输入: {user_input}")
print("="*80)

# ❌ 无脱敏：直接发送给 LLM
print("\n❌ 无脱敏（直接发送）")
print("-"*80)
unsafe_prompt = f"""用户查询：{user_input}

请回答用户的问题。"""

print(f"发送给LLM的Prompt:\n{unsafe_prompt}\n")
print("⚠️ 危害: PII 暴露给 LLM，可能被记录到日志或训练数据")

time.sleep(2)

# ✅ 有脱敏：检测并替换为占位符
print("\n\n✅ 有脱敏（检测并替换）")
print("-"*80)
masked_input, pii_map = detector.detect_and_mask(user_input)

safe_prompt = f"""用户查询：{masked_input}

请回答用户的问题。"""

print(f"检测到的 PII:")
for placeholder, info in pii_map.items():
    print(f"  {placeholder}: {info['value']} ({info['type']})")

print(f"\n脱敏后的输入: {masked_input}")
print(f"\n发送给LLM的Prompt:\n{safe_prompt}")
print("\n✓ 优势: PII 被占位符替代，LLM 无法获取真实敏感信息")

print("\n" + "="*80)
print(f"防护效果: 检测并脱敏了 {len(pii_map)} 个 PII")
print("="*80)

实验一：用户输入的 PII 检测与脱敏
原始输入: 我叫张三，手机号13812345678，邮箱zhangsan@example.com，请帮我查询订单状态

❌ 无脱敏（直接发送）
--------------------------------------------------------------------------------
发送给LLM的Prompt:
用户查询：我叫张三，手机号13812345678，邮箱zhangsan@example.com，请帮我查询订单状态

请回答用户的问题。

⚠️ 危害: PII 暴露给 LLM，可能被记录到日志或训练数据


✅ 有脱敏（检测并替换）
--------------------------------------------------------------------------------
检测到的 PII:
  <PHONE_0>: 13812345678 (phone)
  <EMAIL_1>: 邮箱zhangsan@example.com (email)

脱敏后的输入: 我叫张三，手机号<PHONE_0>，<EMAIL_1>，请帮我查询订单状态

发送给LLM的Prompt:
用户查询：我叫张三，手机号<PHONE_0>，<EMAIL_1>，请帮我查询订单状态

请回答用户的问题。

✓ 优势: PII 被占位符替代，LLM 无法获取真实敏感信息

防护效果: 检测并脱敏了 2 个 PII


---

# 实验二：检索内容的分级脱敏

**场景**：RAG 系统检索到多条文档，包括自己的和他人的

**风险**：检索到的他人文档包含 PII，存在越权访问风险

**策略**：分级脱敏
- 自己的文档：部分脱敏（138****5678）帮助识别
- 他人的文档：完全脱敏（<PHONE_0>）严格保护

In [4]:
# 实验二：检索内容分级脱敏

# 模拟：用户查询
user_query = "查询手机号尾号5678的订单"
current_user_id = "user_001"

# 模拟：向量检索到的文档
retrieved_docs = [
    {
        'content': '订单记录：客户张三，手机13812345678，订单号202401001，已发货',
        'owner_id': 'user_001',  # 当前用户自己的文档
        'source': '订单数据库'
    },
    {
        'content': '订单记录：客户李四，手机13987654321，订单号202401002，待付款',
        'owner_id': 'user_002',  # 他人的文档
        'source': '订单数据库'
    }
]

print("="*80)
print("实验二：检索内容的分级脱敏")
print("="*80)
print(f"用户查询: {user_query}")
print(f"当前用户: {current_user_id}")
print(f"检索到: {len(retrieved_docs)} 条文档")
print("="*80)

# ❌ 无脱敏：直接使用检索内容
print("\n❌ 无脱敏（直接使用）")
print("-"*80)
unsafe_context = "\n".join([doc['content'] for doc in retrieved_docs])
print(f"拼接的上下文:\n{unsafe_context}\n")
print("⚠️ 危害: 他人的 PII (13987654321) 暴露给了当前用户")

time.sleep(2)

# ✅ 分级脱敏
print("\n\n✅ 分级脱敏（区分自己和他人）")
print("-"*80)

masked_docs = []
for doc in retrieved_docs:
    if doc['owner_id'] == current_user_id:
        # 自己的文档：部分脱敏（保留识别信息）
        masked_content = detector.partial_mask(doc['content'])
        level = "部分脱敏"
    else:
        # 他人的文档：完全脱敏（严格保护）
        masked_content, _ = detector.detect_and_mask(doc['content'])
        level = "完全脱敏"

    masked_docs.append(masked_content)
    print(f"文档{len(masked_docs)} [{level}]: {masked_content}")

safe_context = "\n".join(masked_docs)

print(f"\n脱敏后的上下文:\n{safe_context}")
print("\n✓ 优势:")
print("  - 自己的数据部分脱敏：138****5678（可识别）")
print("  - 他人的数据完全脱敏：<PHONE_0>（严格保护）")

print("\n" + "="*80)
print("防护效果: 分级脱敏，平衡可用性和安全性")
print("="*80)

实验二：检索内容的分级脱敏
用户查询: 查询手机号尾号5678的订单
当前用户: user_001
检索到: 2 条文档

❌ 无脱敏（直接使用）
--------------------------------------------------------------------------------
拼接的上下文:
订单记录：客户张三，手机13812345678，订单号202401001，已发货
订单记录：客户李四，手机13987654321，订单号202401002，待付款

⚠️ 危害: 他人的 PII (13987654321) 暴露给了当前用户


✅ 分级脱敏（区分自己和他人）
--------------------------------------------------------------------------------
文档1 [部分脱敏]: 订单记录：客户张三，手机138****5678，订单号202401001，已发货
文档2 [完全脱敏]: 订单记录：客户李四，手机<PHONE_0>，订单号202401002，待付款

脱敏后的上下文:
订单记录：客户张三，手机138****5678，订单号202401001，已发货
订单记录：客户李四，手机<PHONE_0>，订单号202401002，待付款

✓ 优势:
  - 自己的数据部分脱敏：138****5678（可识别）
  - 他人的数据完全脱敏：<PHONE_0>（严格保护）

防护效果: 分级脱敏，平衡可用性和安全性


---

# 实验三：LLM 输出的 PII 检查

**场景**：LLM 可能在输出中意外泄露 PII

**风险**：
1. 模型复现了输入中的 PII
2. 模型幻觉生成了新的 PII

**策略**：输出后处理，检查并二次脱敏

In [5]:
# 实验三：LLM 输出 PII 检查

# 模拟：构造一个可能泄露PII的查询
query_with_pii = "用户张三的手机号是13812345678，请生成一封确认短信"

print("="*80)
print("实验三：LLM 输出的 PII 检查")
print("="*80)
print(f"用户查询: {query_with_pii}")
print("="*80)

# ❌ 无检查：直接返回 LLM 输出
print("\n❌ 无检查（直接返回）")
print("-"*80)
unsafe_prompt = f"""请根据以下信息生成一封确认短信：

{query_with_pii}

短信内容："""

llm_output = get_completion(unsafe_prompt, show_output=True)

# 检查输出是否包含PII
has_pii = bool(re.search(r'1[3-9]\d{9}', llm_output))
print(f"\n⚠️ 检查结果: {'检测到PII泄露' if has_pii else '未检测到PII'}")
if has_pii:
    print(f"   危害: LLM 输出包含手机号，可能被日志记录或展示给用户")

time.sleep(3)

# ✅ 有检查：输出后处理
print("\n\n✅ 有检查（输出后处理）")
print("-"*80)

# 先脱敏输入
masked_query, pii_map = detector.detect_and_mask(query_with_pii)
safe_prompt = f"""请根据以下信息生成一封确认短信：

{masked_query}

重要：不要在输出中包含 <PHONE_0> 的真实值，保持占位符即可。

短信内容："""

llm_output2 = get_completion(safe_prompt, show_output=True)

# 二次检查：防止模型幻觉生成新PII
def check_output_pii(output, pii_map):
    """检查输出是否泄露PII"""
    # 检查1：是否还原了占位符
    for placeholder, info in pii_map.items():
        if info['value'] in output:
            output = output.replace(info['value'], placeholder)
            print(f"\n⚠️ 检测到还原的PII，已重新脱敏: {info['value']} → {placeholder}")

    # 检查2：是否生成了新的PII
    new_masked, new_pii = detector.detect_and_mask(output)
    if new_pii:
        print(f"\n⚠️ 检测到新生成的PII，已脱敏: {list(new_pii.values())}")
        output = new_masked

    return output

safe_output = check_output_pii(llm_output2, pii_map)

print(f"\n最终输出:\n{safe_output}")
print("\n✓ 优势: 双重检查，防止PII泄露")

print("\n" + "="*80)
print("防护效果: 输入脱敏 + 输出检查 = 双重保护")
print("="*80)

实验三：LLM 输出的 PII 检查
用户查询: 用户张三的手机号是13812345678，请生成一封确认短信

❌ 无检查（直接返回）
--------------------------------------------------------------------------------
**确认短信模板：**

尊敬的张三，您好！  
您的手机号 **13812345678** 已成功完成【XXX服务】的确认操作。感谢您的使用！如有疑问，请联系客服热线：XXX-XXXX-XXXX。  

**温馨提示：** 请勿向他人泄露个人信息，谨防诈骗。  

---  
*注：请根据实际业务场景补充【XXX服务】具体内容及客服信息。*

⚠️ 检查结果: 检测到PII泄露
   危害: LLM 输出包含手机号，可能被日志记录或展示给用户


✅ 有检查（输出后处理）
--------------------------------------------------------------------------------
【确认短信】  
尊敬的张三，您好！  
感谢您使用我们的服务。您的手机号为<PHONE_0>，请确认是否正确。如有疑问，请及时联系客服。  
祝您生活愉快！  
[服务团队]

最终输出:
【确认短信】  
尊敬的张三，您好！  
感谢您使用我们的服务。您的手机号为<PHONE_0>，请确认是否正确。如有疑问，请及时联系客服。  
祝您生活愉快！  
[服务团队]

✓ 优势: 双重检查，防止PII泄露

防护效果: 输入脱敏 + 输出检查 = 双重保护


---

# 总结

## RAG 流程的完整 PII 保护方案

```
用户输入
   ↓
[1. 输入脱敏] ← 检测PII，替换为占位符
   ↓
向量检索
   ↓
[2. 权限检查] ← 确认文档归属
   ↓
[3. 分级脱敏] ← 自己的：部分脱敏，他人的：完全脱敏
   ↓
拼接 Prompt
   ↓
LLM 生成
   ↓
[4. 输出检查] ← 二次脱敏，防止泄露
   ↓
返回用户
```

## 脱敏策略总结

| 场景 | 策略 | 示例 | 适用性 |
|------|------|------|--------|
| 用户输入 | 完全脱敏 | 13812345678 → `<PHONE_0>` | 所有场景 |
| 自己的数据 | 部分脱敏 | 13812345678 → 138****5678 | 帮助识别 |
| 他人的数据 | 完全脱敏 | 13987654321 → `<PHONE_1>` | 严格保护 |
| LLM 输出 | 后处理检查 | 检测并重新脱敏 | 防止泄露 |
| 日志记录 | 完全脱敏 | 所有PII替换为占位符 | 合规要求 |

## 工具选择

### 自定义正则（本实验方式）
- **优势**：灵活、轻量、可控
- **劣势**：需要维护正则规则
- **适用**：中小型项目

### 开源工具
- **Presidio**（微软）：支持多语言PII检测
- **AWS Comprehend**：云服务，准确率高
- **Google DLP API**：企业级方案

## 关键原则

1. **分层防护**：输入 + 检索 + 输出三层拦截
2. **分级脱敏**：根据数据归属和场景选择策略
3. **最小暴露**：只在必要时保留部分信息
4. **持续监控**：日志审计，发现新型PII泄露

## 注意事项

❌ **常见误区**：
- 只脱敏输入，忽略检索内容
- 所有数据用同一策略（过度或不足）
- 不检查LLM输出（模型可能泄露）

✅ **最佳实践**：
- 完整流程保护，不留死角
- 分级脱敏，平衡可用性和安全性
- 定期审计，及时发现问题

**结论**：PII 保护不是一劳永逸，需要技术+流程+监控的组合拳